<a href="https://colab.research.google.com/github/AlkaidCheng/example/blob/master/CatBoostHParamTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install  --upgrade bayesian-optimization
! pip install ray[tune]

     |████████████████████████████████| 19.1MB 156kB/s 
     |████████████████████████████████| 1.2MB 49.5MB/s 
     |████████████████████████████████| 2.9MB 47.6MB/s 
     |████████████████████████████████| 71kB 11.9MB/s 
     |████████████████████████████████| 204kB 41.6MB/s 
     |████████████████████████████████| 256kB 51.2MB/s 
     |████████████████████████████████| 153kB 59.7MB/s 
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl size=3162 sha256=40952fddaee282afe6618f664dc1b4f8b18dd66a9f7c3d7a6374b7ff90f85a85
  Stored in directory: /root/.cache/pip/wheels/d3/00/b3/32d613e19e08a739751dd6bf998cfed277728f8b2127ad4eb7
Successfully built idna-ssl


In [2]:
!wget https://gitlab.cern.ch/clcheng/mlhep-googlesummerofcode/raw/master/Prerequisite/MachineLearning/QIS_EXAM_200Events.npz

--2020-03-08 05:57:31--  https://gitlab.cern.ch/clcheng/mlhep-googlesummerofcode/raw/master/Prerequisite/MachineLearning/QIS_EXAM_200Events.npz
Resolving gitlab.cern.ch (gitlab.cern.ch)... 188.184.30.144, 188.184.30.115, 188.184.31.109, ...
Connecting to gitlab.cern.ch (gitlab.cern.ch)|188.184.30.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9140 (8.9K) [application/zip]
Saving to: ‘QIS_EXAM_200Events.npz’

QIS_EXAM_200Events. 100%[===================>]   8.93K  --.-KB/s    in 0s      

2020-03-08 05:57:32 (209 MB/s) - ‘QIS_EXAM_200Events.npz’ saved [9140/9140]



In [0]:
import numpy as np
def shuffle_zippedarrays(arrays):
  shape = arrays[0].shape
  assert all(shape == arrays[0].shape for arr in arrays)
  index = np.random.permutation(shape[0])
  return [arr[index] for arr in arrays]

def shuffle_data(input, label):
  output = shuffle_zippedarrays([input,label])
  return output[0], output[1]

def load_data(arrays, shuffle = True):
  data = {}
  for key in arrays:
    input, label = np.array([]), np.array([])
    for klabel in arrays[key].item():
      _input = arrays[key].item()[klabel]
      _label = np.full((_input.shape[0],),int(klabel))
      input = np.concatenate((input,_input),axis=0) if input.size else _input
      label = np.concatenate((label,_label),axis=0) if label.size else _label
    if shuffle:
      input, label = shuffle_data(input, label)
    data[key] = {'input': input, 'label': label}
  return data
def load_train_test_input_labels(arrays, shuffle = True):
  data = load_data(arrays, shuffle)
  return data['training_input']['input'], data['test_input']['input'], data['training_input']['label'], data['test_input']['label']

In [0]:
data = np.load('QIS_EXAM_200Events.npz',allow_pickle=True)
train_input, test_input, train_label, test_label = load_train_test_input_labels(data)
X = np.concatenate((train_input,test_input),axis = 0)
Y = np.concatenate((train_label,test_label),axis = 0)

In [5]:
! pip install catboost

     |████████████████████████████████| 64.4MB 46kB/s 


In [15]:
import catboost as cb
import numpy as np
import sklearn.datasets
import sklearn.metrics
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.schedulers import AsyncHyperBandScheduler
import ray
from hyperopt import hp
from ray import tune
from pdb import set_trace
def train_breast_cancer(params):
    dataset = cb.Pool(X, Y)
    scores = cb.cv(
            pool=dataset,
            params=params,
            fold_count=2,
            verbose=False)
        
    # scores returns a dictionary with mean and std (per-fold) of metric 
    # value for each cv iteration, we choose minimal value of objective 
    # mean (though it will be better to choose minimal value among all folds)
    # because noise is additive
    max_mean_auc = np.max(scores['test-Logloss-mean'])
    tune.track.log(
        mean_accuracy=max_mean_auc,
        done=True)
    
ray.shutdown()
ray.init(webui_host='127.0.0.1')


space = {
    'iterations': hp.quniform('iterations', 10, 1000, 1),
    'depth': hp.quniform('depth', 1, 10, 1),
    'learning_rate': hp.loguniform('learning_rate', 1e-5, np.log(0.2)),
    'random_strength': hp.loguniform('random_strength', 1e-7, 10),
    'bagging_temperature': hp.uniform('bagging_temperature', 0.0, 1.0),
    'border_count': hp.quniform('border_count', 1, 255, 1),
    'l2_leaf_reg': hp.quniform('l2_leaf_reg', 2,30 , 1),
    "loss_function": "Logloss",
    'task_type' : 'GPU'
}


algo = HyperOptSearch(
        space,
        max_concurrent=4,
        metric="mean_accuracy",
        mode="min")

from ray.tune.schedulers import ASHAScheduler
! rm -r /lightgbm_result
analysis = tune.run(
    train_breast_cancer,
    num_samples=10,
    verbose =  0,
    resources_per_trial={'gpu': 1},
    search_alg = algo,
    local_dir = '/lightgbm_result',
    scheduler=AsyncHyperBandScheduler(metric="mean_accuracy", mode="min"))
print("Best config: ", analysis.get_best_config(metric="mean_accuracy"))

2020-03-08 06:25:14,051	WARNING services.py:586 -- setpgrp failed, processes may not be cleaned up properly: [Errno 1] Operation not permitted.
2020-03-08 06:25:14,053	INFO resource_spec.py:212 -- Starting Ray with 6.59 GiB memory available for workers and up to 3.3 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-03-08 06:25:14,362	INFO services.py:1078 -- View the Ray dashboard at 127.0.0.1:8265
2020-03-08 06:25:15,664	INFO function_runner.py:250 -- tune.track signature detected.
2020-03-08 06:26:24,523	WARNING worker.py:1058 -- The actor or task with ID ffffffffffffffff34c643a70100 is pending and cannot currently be scheduled. It requires {CPU: 1.000000}, {GPU: 1.000000} for execution and {CPU: 1.000000}, {GPU: 1.000000} for placement, but this node only has remaining {GPU: 1.000000}, {node:172.28.0.2: 1.000000}, {CPU: 2.000000}, {memory: 6.591797 GiB}, {object_store_memory: 2.246094 GiB}. In total there are 0 pending ta

Best config:  {'bagging_temperature': 0.4804469429091168, 'border_count': 19.0, 'depth': 3.0, 'iterations': 726.0, 'l2_leaf_reg': 18.0, 'learning_rate': 0.9023050715669971, 'loss_function': 'Logloss', 'random_strength': 1267.4156381989217}
